## Assignment Questions 

## Random Forest Classifier With Pipeline And Hyperparameter Tuning

In [2]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [6]:
df.time.unique()

array([0, 1])

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [8]:
## independent and dependent feature
X=df.drop(labels=['time'],axis=1)
y=df.time

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## HAndle Missing Values
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer

In [11]:
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']

In [12]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# X_train and y_train are your training data and labels

# Create a random forest classifier
rf_classifier = RandomForestClassifier()

# Create the RFE object and specify the number of features to select
rfe = RFE(estimator=rf_classifier, n_features_to_select=10)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[rfe.support_]

ValueError: could not convert string to float: 'Male'

In [13]:
## feature Engineering Automation
## Numerical Pipelines
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ##Missing Values
        ('scaler',StandardScaler()) ## feature Scaling
    ]

)

#categorical Pipeline
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), ## handling Missing values
                ('onehotencoder',OneHotEncoder()) ## Categorical features to numerical
                ]

            )  


In [14]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)

])

In [15]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# y_train and y_test are your training and test labels

# Create the random forest classifier
rf_classifier = RandomForestClassifier()

# Fit the classifier on the preprocessed training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the preprocessed test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9591836734693877


In [20]:
# Encode the categorical columns using one-hot encoding
ct = ColumnTransformer(
    [("encoder", OneHotEncoder(),categorical_cols)],
    remainder="passthrough"
)
X_encoded = ct.fit_transform(X)

# Create the Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Create the RFE object and specify the number of features to select
rfe = RFE(estimator=rf_classifier, n_features_to_select=10)

# Fit RFE on the encoded features and target variable
rfe.fit(X_encoded, y)

# Get the selected features
selected_features = ct.get_feature_names_out()[rfe.support_]
print(selected_features)

['encoder__sex_Female' 'encoder__sex_Male' 'encoder__smoker_No'
 'encoder__day_Fri' 'encoder__day_Sat' 'encoder__day_Sun'
 'encoder__day_Thur' 'remainder__total_bill' 'remainder__tip'
 'remainder__size']


__Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then
use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its
accuracy.__

In [1]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the individual classifiers
rf_classifier = RandomForestClassifier()
lr_classifier = LogisticRegression()

# Create the voting classifier
voting_classifier = VotingClassifier(
    estimators=[('rf', rf_classifier), ('lr', lr_classifier)],
    voting='hard'  # Use majority voting
)

# Train the pipeline
voting_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = voting_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
